<a href="https://colab.research.google.com/github/Regina-Arthur/Python-Coding-Projects/blob/main/Pytorch_tutorial/Resnet_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import  Subset
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
class Bottleneck(nn.Module):
    expansion = 4  # last Conv expands channels by 4×

    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super().__init__()
        # 1x1 Conv (reduce dimensions)
        self.conv1 = nn.Conv2d(in_channels= in_channels, out_channels= out_channels, kernel_size=1, stride=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)

        # 3x3 Conv (main spatial conv)
        self.conv2 = nn.Conv2d(in_channels= out_channels, out_channels= out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        # 1x1 Conv (expand back up)
        self.conv3 = nn.Conv2d(in_channels= out_channels, out_channels= out_channels * self.expansion, kernel_size=1, stride=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels * self.expansion)

        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample  # used if input/output dims don’t match

    def forward(self, x):
        identity = x  # save input for skip connection

        out = self.relu(self.bn1(self.conv1(x)))
        out = self.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))

        # adjust dimensions if needed
        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity  # skip connection
        out = self.relu(out)

        return out


class ResNet50(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()

        # Initial conv layer
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Residual stages
        self.layer1 = self._make_layer(in_channels= 64, out_channels= 64, blocks=3, stride=1)   # 256 out
        self.layer2 = self._make_layer(in_channels= 256, out_channels= 128, blocks=4, stride=2) # 512 out
        self.layer3 = self._make_layer(in_channels= 512, out_channels= 256, blocks=6, stride=2) # 1024 out
        self.layer4 = self._make_layer(in_channels= 1024, out_channels= 512, blocks=3, stride=2)# 2048 out

        # Final classification head
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))  # Global AvgPool
        self.fc = nn.Linear(512 * Bottleneck.expansion, num_classes)

    def _make_layer(self, in_channels, out_channels, blocks, stride):
        """Builds one stage with multiple bottleneck blocks"""
        downsample = None
        if stride != 1 or in_channels != out_channels * Bottleneck.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * Bottleneck.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * Bottleneck.expansion)
            )

        layers = []
        layers.append(Bottleneck(in_channels, out_channels, stride, downsample))
        for _ in range(1, blocks):
            layers.append(Bottleneck(out_channels * Bottleneck.expansion, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        # Initial
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)

        # Stages
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        # Head
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# # from google.colab import drive
# # drive.mount('/content/drive')

# zip_file_path = '/content/drive/My Drive/Concrete.rar'  # Replace with your zip file's path
# extract_path = '/content/drive/My Drive/concrete_data'  # Desired directory for extracted files

# import os
# os.makedirs(extract_path, exist_ok=True)

# !apt-get install unrar
# !unrar x -o+ "$zip_file_path" "$extract_path"

In [11]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.PILToTensor(),
    transforms.ConvertImageDtype(torch.float)
])

data = "/content/drive/My Drive/concrete_data"

dataset = datasets.ImageFolder(root= data, transform= transform)
print(dataset.classes, dataset.class_to_idx)

# Get all labels (targets)
labels = [label for _, label in dataset.samples]  # list of class indices
labels = np.array(labels)

# First split train+val vs test
train_val_idx, test_idx = train_test_split(
    np.arange(len(labels)),
    test_size=0.15,
    stratify=labels,
    random_state=42
)

# Now split train vs val
train_idx, val_idx = train_test_split(
    train_val_idx,
    test_size=0.15,
    stratify=labels[train_val_idx],
    random_state=42
)


train= Subset(dataset, train_idx)
val = Subset(dataset, val_idx)
test = Subset(dataset, test_idx)

# image, label = train[0]
# print(image.shape, label)

train_loader = torch.utils.data.DataLoader(train, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size=32, shuffle=False)

['Negative', 'Positive'] {'Negative': 0, 'Positive': 1}


In [ ]:
model = ResNet50()
model.to(device)

loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)


epochs = 100
for epoch in range(epochs):
  model.train()
  running_loss = 0.0
  correct = 0
  total = 0

  for images, labels in train_loader:
    images, labels = images.to(device), labels.to(device)

    optimizer.zero_grad()
    outputs = model(images)
    train_loss = loss(outputs, labels)
    train_loss.backward()
    optimizer.step()
    running_loss += train_loss.item()
    _, predicted = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

  train_loss = running_loss/ len(train_loader)
  train_accuracy = 100* correct/total

  print(f"Epoch {epoch+1} / {epochs}, train loss = {train_loss}, train accuracy = {train_accuracy}")

for epoch in range(epochs):
  model.eval()
  running_loss = 0.0
  correct = 0
  total = 0

  for images, labels in val_loader:
    images, labels = images.to(device), labels.to(device)

    outputs = model(images)
    val_loss = loss(outputs, labels)
    running_loss += val_loss.item()
    _, predicted = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

  val_loss = running_loss/ len(train_loader)
  val_accuracy = 100* correct/total

  print(f"Epoch {epoch+1}/{epochs} | "
            f"Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}% | "
            f"Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}%")

for epoch in range(epochs):
  model.eval()
  running_loss = 0.0
  correct = 0
  total = 0

  for images, labels in test_loader:
    images, labels = images.to(device), labels.to(device)

    outputs = model(images)
    test_loss = loss(outputs, labels)
    running_loss += test_loss.item()
    _, predicted = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

  test_loss = running_loss/ len(train_loader)
  test_accuracy = 100* correct/total

  print(f"Epoch {epoch+1}/{epochs} | "
            f"Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}% | "
            f"Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}%"
            f"Test loss: {test_loss:.4f}, Test Acc: {test_accuracy:.2f}%")